In [ ]:
import os
import re
import time
import requests
from pprint import pprint
from zhconv import convert

In [ ]:
sleep_time=0.2 #temps du scraping
#étape 1 : chercher les inscriptions correspondant
text = 'mot_clé'
time_start=time.time()
with open('library.txt','r',encoding='utf-8') as f: #crée un fichier d'enregistrement
    library=f.read()

#données de formulaire pour les demandes
datals={'ConditionKey2': 'YW','ConditionValue2': '','SubConditionQuery0': 2,'ConditionKey': 'SW','ConditionValue': Keyword,
    'SubConditionQuery': 2,'beforeQueryValue': 2,'beforeConditionValue': Keyword,'beforeConditionValue2': '','beforeConditionKey': 'SW',
    'searchFieldJG': '','beforecheckedCatID': library,'beforeselectedCatTypeID': 2,'fieldName': '','fieldValue': '','currentSubid': 0,
    'checkedCatID': library,'selectedCatTypeID': 2,'pageIndexNow': 1,'sortField': '','sortFieldBefore': '','sortInit': '',
    'sortDescAsc': '','listOrabst': '','dbID': 1,'dbName': 'BONE','MediaType': 5,'topicCLSCode': '','topicCLSNodeCode': '',
    'AutoLoad': True,'SearchFrom': 'SingleSearch','pageSize': 12,'swid': '','swid': '','SubConditionType0': 0,'SubConditionKey0': 'PM',
    'SubConditionValue0': '','X-Requested-With': 'XMLHttpRequest'}
headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}

#obtenir le nombre total de pages et le nombre total d'entrées
search_code=requests.post(url='http://jgw.aynu.edu.cn/AyjgwSingleSearch/Search',data=datals,headers=headers).text
# print(search_code)
search_info=re.compile(r'<span>(.*?)条结果.*?/(.*?)</span>',re.S).findall(search_code)

#scraping par page
Data=[] #créer une liste vide pour enregistrer les données
Count=1

#maintenant on a une boucle utilisée pour effectuer le web scraping d'une liste de pages
for page in range(pages): #pour chaque page
    current_page = pages +1

        if current_page==1: #si la page actuelle est la première page ? Si oui, la variable search_code_child est définie par search_code
            search_code_child=search_code
        else: #sinon, requête POST vers l'URL
            datals['pageIndexNow']=current_page
            search_code_child=requests.post(url='http://jgw.aynu.edu.cn/AyjgwSingleSearch/Search',data=datals,headers=headers).text
        search_result=re.compile(r'''<div class='tc mb10'>.*?<a onclick="confirmDialog(.*?)" href=".*?">(.*?)</a>''',re.S).findall(search_code_child)
        #regex pour chercher les liens du site + le nom
        for i in search_result:
            Dict={'Count':Count} #créer un dictionnaire pour se souvenir des lignes déjàs prises
            temp=eval(i[0]) #fomer un tuple avec le numéro inventaire
            URL=f'http://jgw.aynu.edu.cn/ajaxpage/home2.0/d/view.html?dbID={temp[0]}&dbName={temp[1]}&DisplayDBName={temp[2]}&sysID={temp[3]}&drnext={temp[4]}' # 详情页url
            Dict['URL']=URL
            BoneID=i[1]
            for j in ['\r','\n','\t',' ']:
                BoneID=BoneID.replace(j,'')
            Dict['BoneID']=BoneID
            Data.append(Dict)
            Count+=1

    time.sleep(sleep_time) #pause pour ralentir et éviter la masse d'information successive


Data = [item for item in Data if not item["BoneID"]=='']
 #supprimer les éléments qui ont un BoneID vide afin d'appliquer correctement la fonction SplitBoneID(Dict)

#étape 2 : chercher les transcriptions
def SplitBoneID(Dict): #enregistrer le numéro d'inventaire
    BoneID=Dict['BoneID']
    tempBoneID=BoneID
    while not tempBoneID[-1].isdigit(): #supprimer les caractères non numériques à la fin de la chaîne
        tempBoneID=tempBoneID[:-1]
    Book,numID='',''
    for i in tempBoneID:
        numID+=i if i.isdigit() else ''
        Book+=i if not i.isdigit() else ''
        #à chaque itération, si le caractère est un chiffre, il est ajouté à la chaîne numID. Sinon, il est ajouté à la chaîne Book
    while numID[0]=='0': #supprimer les 0 qui ne servent à rien
        numID=numID[1:]
    numID=eval(numID)
    return Book,numID
#La fonction renvoie les valeurs sous la forme d'un tuple (Book, numID)

#maintenant on extrait les transcriptions
#en fait, le numéro inventaire est composé de l'abbréviation du recueil + numéro
def GetText(Data):
    for Dict in Data:
        Book,numID=SplitBoneID(Dict)[0],SplitBoneID(Dict)[1]
        if Book in ['合','合补','屯南','花东']: #le titre des recueils
            try:
                if Book=='合':
                    bhfl=1
                elif Book=='合补':
                    bhfl=2
                elif Book=='屯南':
                    bhfl=3
                elif Book=='':
                    bhfl=4
                    gxdsURL=f'http://www.guoxuedashi.net/jgwhj/?bh={numID}&bhfl={bhfl}'  #nouveau URL

                #regex pour le contenu de la page html
                obj_gxds=re.compile(r'<tr><td width=.*?>(.*?)</td><td width=.*?>'
                        r'(.*?)</td><td >(.*?)</td><td width=.*?>.*?</td></tr>',re.S)
                result_gxds=obj_gxds.findall(requests.get(gxdsURL).text)
                if not result_gxds==[]:
                    if len(result_gxds)==1: #si une seule correspondance trouvée
                        Text=result_gxds[0][2]
                        Text=convert(Text,'zh-hans') #converti en chinois simplifié
                        Dict['Text']=Text #enregistré le texte à la clé Text
                    else: #si plusieurs correspondances
                        gxdsID=[]; Text=''
                        for r in result_gxds:
                            gxdsID.append(r[0])
                        if len(set(gxdsID))==1: #si les id sont les même
                            for r in result_gxds:
                                Text+=f'{r[1]}：{r[2]}' #les mettre ensemble
                                Text+='<br />' if not r==result_gxds[-1] else ''
                        else: #sinon, ils n'ont pas les mêmes id
                            for r in result_gxds: #les mettres ensemble mais ajouter aussi les id
                                Text+=f'{r[0]}-{r[1]}：{r[2]}'
                                Text+='<br />' if not r==result_gxds[-1] else ''
                        Text=convert(Text,'zh-hans')
                        Dict['Text']=Text #ajouté le texte
                else: #s'il y en a pas
                    Dict['Text']=''
            except:
                Dict['Text']='Error!'
        else:
            Dict['Text']=''
        Text=Dict['Text']
        if Text !='':
            if Text[0].isalpha() or Text[-1].isdigit() or Text[0]=='<' or Text[-1]=='>' or 'a href' in Text:
                Dict['Text']=''

        time.sleep(sleep_time)

# étape 3 : extraire les métadonnées
def GetPeriod(Data): #les périodes ou les prêtres en fonction des différents recueils
    for Dict in Data:
        Book,numID=SplitBoneID(Dict)[0],SplitBoneID(Dict)[1]
        if Book=='合':
            if 1<=numID<=19753 or 39477<=numID<=40814:
                Period='第一期'
            elif 19754<=numID<=22536 or 40815<=numID<=40910:
                Period='第一期附'
            elif 22537<=numID<=26878 or 40911<=numID<=41302:
                Period='第二期'
            elif 26879<=numID<=31968 or 41303<=numID<=41453:
                Period='第三期'
            elif 31969<=numID<=35342 or 41454<=numID<=41694:
                Period='第四期'
            elif 35343<=numID<=39476 or 41695<=numID<=41956:
                Period='第五期'
            else:
                Period='Error!'
        elif Book=='合补':
            if 1<=numID<=6543 or 13171<=numID<=13259:
                Period='第一期'
            elif 6544<=numID<=6954 or 13260<=numID<=13269:
                Period='第一期附'
            elif 6955<=numID<=8705 or 13270<=numID<=13326:
                Period='第二期'
            elif 8706<=numID<=10409 or 13327<=numID<=13383:
                Period='第三期'
            elif 10410<=numID<=10940 or 13384<=numID<=13422:
                Period='第四期'
            elif 10941<=numID<=13170 or 13423<=numID<=13450:
                Period='第五期'
            else:
                Period='Error!'
        elif Book=='旅藏':
            if 1<=numID<=1331:
                Period='第一期'
            elif 1332<=numID<=1753:
                Period='第二期'
            elif 1754<=numID<=1860:
                Period='第三期'
            elif 1861<=numID<=1912:
                Period='第四期'
            elif 1913<=numID<=2211:
                Period='第五期'
            else:
                Period='Error!'
        elif Book=='卡':
            if 1<=numID<=12:
                Period='师组'
            elif numID==13:
                Period='丙种子卜辞（子组）'
            elif 14<=numID<=17:
                Period='师宾间组'
            elif 18<=numID<=70:
                Period='师历间组'
            elif 71<=numID<=95:
                Period='宾组'
            elif 96<=numID<=106:
                Period='历组'
            elif 107<=numID<=364:
                Period='出组'
            elif 365<=numID<=366:
                Period='历无'
            elif 367<=numID<=379:
                Period='何组'
            elif 380<=numID<=393:
                Period='无名组'
            elif 394<=numID<=402:
                Period='习刻'
            elif 403<=numID<=404:
                Period='伪刻'
            elif 405<=numID<=406:
                Period='遗失甲骨'
            elif 407<=numID<=609:
                Period='护身符'
            else:
                Period='Error!'
        else:
            Period=''
        Dict['Period']=Period


t2=time.time()
GetText(Data)

t3=time.time()
GetPeriod(Data)


#étape 4 : enregistrer
def SaveToMd():
    with open (savepath,'w',encoding='utf-8') as f: #créer un nouveau document
        f.write(f'## {Keyword}（{len(Data)}/{items}）\n\n') #mot_clé et le nombre des inscriptions trouvées
        f.write('\n\n')
        f.write('|      | id | période | inscriptions |\n') #construire un tableau
        f.write('| ---- | ---- | ---- | ---- |\n')
        for Dict in Data: #ajouter les données une par une
            Count=Dict['Count']
            BoneID=Dict['BoneID']
            URL=Dict['URL']
            Period=Dict['Period']
            Text=Dict['Text']

            f.write(f'| **{Count}** | [**{BoneID}**]({URL}) | {Period} | {Text}  |\n')

savepath=f'{Keyword}.md'
SaveToMd() #enregistrer dans l'ordinateur
os.system(savepath)
